In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pennylane as qml
import tensorflow as tf
from tensorflow.keras.layers import Lambda, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_curve, auc
from pennylane.templates import AmplitudeEmbedding, StronglyEntanglingLayers

# --- 1. Data Collection and Preparation ---

# Hyperparameters for model setup
CSV_PATH   = "C:\\Users\\fenlei\\Downloads\\EEG data.xlsx"
N_QUBITS   = 2             # 2**4 = 16 amplitudes
AMP_DIM    = 2 ** N_QUBITS
N_LAYERS   = 3
BATCH_SIZE = 2
EPOCHS     = 20
VAL_SPLIT  = 0.1

# Read the file
df = pd.read_excel(CSV_PATH)
df = df.rename(columns={
    'коды': 'Subject_Code',
    'группа': 'Group',
    'нет': 'Electrode_Pair',
    'отведения': 'Brain_Region',
    'ритм': 'EEG_Band',
    'observ': 'Mean_Obs_Amplitude',
    'imagin': 'Mean_Imag_Amplitude',
    'execut': 'Mean_Exec_Amplitude',
    'together': 'Mean_Total_Amplitude',
    'observ.1': 'Z_Obs_Amplitude',
    'imagin.1': 'Z_Imag_Amplitude',
    'execut.1': 'Z_Exec_Amplitude',
    'together.1': 'Z_Total_Amplitude'
})

# --- 2. Map 'Group' to anxiety labels (0 = control, 1 = patient/anxiety) ---
df['Anxiety_Level'] = df['Group'].map({'control': 0, 'patients': 1})

# --- 3. Pivot so each subject is one feature vector ---
features = [
    'Mean_Obs_Amplitude', 'Mean_Imag_Amplitude',
    'Mean_Exec_Amplitude', 'Mean_Total_Amplitude',
    'Z_Obs_Amplitude', 'Z_Imag_Amplitude',
    'Z_Exec_Amplitude', 'Z_Total_Amplitude'
]
pivot = df.pivot_table(index='Subject_Code', columns='EEG_Band', values=features)
# Correctly flatten MultiIndex columns to "feature_band"
pivot.columns = [f"{feature}_{band}" for (band, feature) in pivot.columns]
X = pivot.values
y = df.drop_duplicates('Subject_Code').set_index('Subject_Code')['Anxiety_Level'].loc[pivot.index].values

# Handle missing values
imputer = KNNImputer(n_neighbors=5)
X_imputed = imputer.fit_transform(X)

# --- 4. Dimensionality reduction & normalization ---
pca = PCA(n_components=4)
X_pca = pca.fit_transform(X_imputed)
X_norm = normalize(X_pca)

# --- 5. Custom train-test split ensuring class balance ---
class_A_indices = np.where(y == 0)[0]
class_B_indices = np.where(y == 1)[0]

train_A_size = 10
train_B_size = 36
test_A_size = 12
test_B_size = 4

train_A_indices = np.random.choice(class_A_indices, size=train_A_size, replace=False)
train_B_indices = np.random.choice(class_B_indices, size=train_B_size, replace=False)

remaining_A = np.setdiff1d(class_A_indices, train_A_indices)
remaining_B = np.setdiff1d(class_B_indices, train_B_indices)

test_A_indices = np.random.choice(remaining_A, size=test_A_size, replace=False)
test_B_indices = np.random.choice(remaining_B, size=test_B_size, replace=False)

train_indices = np.concatenate([train_A_indices, train_B_indices])
test_indices = np.concatenate([test_A_indices, test_B_indices])

X_train, y_train = X_norm[train_indices], y[train_indices]
X_val, y_val = X_norm[test_indices], y[test_indices]

class_weight = compute_class_weight("balanced", classes=np.unique(y_train), y=y_train)
class_weight = dict(enumerate(class_weight))

# --- 6. defining a Qunatum Layer ---
class QNNLayer(tf.keras.layers.Layer):
    def __init__(self, n_qubits, n_layers):
        super(QNNLayer, self).__init__()
        # Number of qubits and quantum layers
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        # Initialize trainable quantum weights
        # Shape: (n_layers, n_qubits, 3) -> 3 angles for RX, RY, RZ rotations
        self.qweights = tf.Variable(
            initial_value=tf.random.uniform(shape=(n_layers, n_qubits, 3), minval=0, maxval=2*np.pi),
            trainable=True
        )
        # Define a quantum device (simulator) with the specified number of qubits
        dev = qml.device("default.qubit", wires=n_qubits)
        
        @qml.qnode(dev, interface="tf")
        def circuit(inputs, weights):
            # Embed classical data into quantum state using amplitude embedding
            AmplitudeEmbedding(inputs, wires=range(n_qubits), normalize=True)
            # Apply strongly entangling layers with the given weights
            StronglyEntanglingLayers(weights, wires=range(n_qubits))
            # Measure the expectation value of Pauli-Z for each qubit
            return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]
        
        self.circuit = circuit

    def call(self, inputs):
        batch_size=BATCH_SIZE
        outputs = []
        for i in range(batch_size):
            single_input = inputs[i]
            output = self.circuit(single_input, self.qweights)
            outputs.append(output)
        return tf.stack(outputs)


# 7. Building a Hybrid Classical-Quantum Model
#       a. Classical layers preprocess EEG features.
#       b. Quantum layer (QNNLayer) transforms features with quantum operations.
#       c. Final classical layer makes the binary prediction. 
inp = tf.keras.Input(shape=(AMP_DIM,))
x = tf.keras.layers.Dense(32, activation="relu")(inp)
x = Dropout(0.2)(x)
x = tf.keras.layers.Dense(AMP_DIM)(x)
x = QNNLayer(N_QUBITS, N_LAYERS)(x)
out = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inp, out)
# Set optimizer with a custom learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)
# Compile the model with binary crossentropy loss, accuracy metric, and AUC metric
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy", tf.keras.metrics.AUC(name="auc")])

# --- 8. Training the model ---
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_val = tf.convert_to_tensor(X_val, dtype=tf.float32)
y_val = tf.convert_to_tensor(y_val, dtype=tf.float32)

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=EPOCHS, batch_size=BATCH_SIZE, class_weight=class_weight)

# Evaluate the model
test_loss, test_acc, test_auc = model.evaluate(X_val, y_val)
print(f"Test Accuracy: {test_acc:.4f}, Test AUC: {test_auc:.4f}")

# --- 9. Evaluate the Result ---
y_pred = model.predict(X_val).flatten()
fpr, tpr, _ = roc_curve(y_val, y_pred)
roc_auc = auc(fpr, tpr)
print(f"ROC AUC: {roc_auc:.4f}")

# --- 10. Save the Model ---
model.save("QML_improved.h5")

Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.4394 - auc: 0.3418 - loss: 0.6269 - val_accuracy: 0.5625 - val_auc: 0.5104 - val_loss: 0.7046
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4156 - auc: 0.3815 - loss: 0.6784 - val_accuracy: 0.5000 - val_auc: 0.4896 - val_loss: 0.6999
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4241 - auc: 0.5412 - loss: 0.7256 - val_accuracy: 0.5000 - val_auc: 0.4375 - val_loss: 0.6951
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3280 - auc: 0.3816 - loss: 0.6856 - val_accuracy: 0.5000 - val_auc: 0.4792 - val_loss: 0.6949
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5579 - auc: 0.6982 - loss: 0.6750 - val_accuracy: 0.6250 - val_auc: 0.5208 - val_loss: 0.6950
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4517 - auc: 0.5065 - loss: 0.7353 - val_accuracy: 0.5625 - val_auc: 0.5417 - val_loss: 0.6905
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step -

InvalidArgumentError: Graph execution error:

Detected at node compile_loss/binary_crossentropy/logistic_loss/mul defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\fenlei\myenv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "c:\Users\fenlei\myenv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\fenlei\myenv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Users\fenlei\myenv\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\fenlei\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 641, in run_forever

  File "C:\Users\fenlei\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 1987, in _run_once

  File "C:\Users\fenlei\AppData\Local\Programs\Python\Python312\Lib\asyncio\events.py", line 88, in _run

  

In [7]:
# --- choose the data you really want to test on -----------------
X_test, y_test = X_val, y_val        # <- replace with the real test set
# ----------------------------------------------------------------

test_loss, test_acc, test_auc = model.evaluate(
    X_test, y_test, batch_size=2, verbose=0)

print(f"Test Accuracy: {test_acc:.4f} | Test AUC: {test_auc:.4f}")

# --- ROC curve ---------------------------------------------------
y_pred = model.predict(X_test, batch_size=2).ravel()   # shape (46,)
fpr, tpr, _ = roc_curve(y_test, y_pred)
print(f"ROC AUC: {auc(fpr, tpr):.4f}")


Test Accuracy: 0.5625 | Test AUC: 0.5833
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step  
ROC AUC: 0.5833
